In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
# mpl.style.use('ggplot')

In [92]:
car = pd.read_csv('quikr_car.csv')

In [93]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [94]:
car.shape

(892, 6)

In [95]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [96]:
backup = car.copy()

## Data Quality
- names are pretty inconsistent
- names have company names attached to it
- some names are spam like 'Maruti Ertiga showroom condition with' and 'Well mentained Tata Sumo'
- company: many of the names are not of any company like 'Used', 'URJENT', and so on.
- year has many non-year values
- year is in object. Change to integer
- Price has Ask for Price
- Price has commas in its prices and is in object
- kms_driven has object values with kms at last.
- It has nan values and two rows have 'Petrol' in them
- fuel_type has nan values

# Cleaning Data

#### Year has many non-year valus

In [97]:
car = car[car['year'].str.isnumeric()]

#### Year is in object. change to intger

In [98]:
car['year'] = car['year'].astype(int)

#### Price has "Ask for Price"

In [99]:
car = car[car['Price'] != 'Ask For Price']

In [100]:
# car['Price'].unique()

#### Price has commas in its prices and is in object


In [101]:
car['Price']=car['Price'].str.replace(',','').astype(int)

#### kms_driven has object values with kms at last

In [102]:
car['kms_driven']=car['kms_driven'].str.split().str.get(0).str.replace(',','')

#### It has nan value values and two rows have  'Petrol' in them

In [103]:
car=car[car['kms_driven'].str.isnumeric()]

In [104]:
car['kms_driven']=car['kms_driven'].astype(int)

#### fuel_type has nan values

In [105]:
car=car[~car['fuel_type'].isna()]

In [106]:
car.shape

(816, 6)

#### name and company had spammed data...but with previous cleaning, thoserows got removed.

##### company does not need any cleaning now. WChwanging car names. keeping only the first 3 words

In [107]:
car['name']=car['name'].str.split().str.slice(start=0, stop=3).str.join(' ')

#### Resetting the index of the final cleaned data

In [108]:
car=car.reset_index(drop=True)

#### Cleaned Data

In [109]:
car.to_csv('Cleaned_Car_data.csv')

In [110]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        816 non-null    object
 1   company     816 non-null    object
 2   year        816 non-null    int32 
 3   Price       816 non-null    int32 
 4   kms_driven  816 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 28.8+ KB


In [111]:
car.describe(include='all')

,name,company,year,Price,kms_driven,fuel_type
count,816,816,816.000000,8.160000e+02,816.000000,816
unique,254,25,NaN,NaN,NaN,3
top,Maruti Suzuki Swift,Maruti,NaN,NaN,NaN,Petrol
freq,51,221,NaN,NaN,NaN,428
mean,NaN,NaN,2012.444853,4.117176e+05,46275.531863,NaN
std,NaN,NaN,4.002992,4.751844e+05,34297.428044,NaN
min,NaN,NaN,1995.000000,3.000000e+04,0.000000,NaN
25%,NaN,NaN,2010.000000,1.750000e+05,27000.000000,NaN
50%,NaN,NaN,2013.000000,2.999990e+05,41000.000000,NaN
75%,NaN,NaN,2015.000000,4.912500e+05,56818.500000,NaN


In [112]:
car=car[car['Price']<6000000]

## Extracting Data

In [113]:
x=car.drop(columns='Price')
y=car['Price']

In [114]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


In [115]:
''' 
# Categorical data are variables that contain label values rather than numeric values.
# Some algorithms can work with categorical data directly.
# For example, a decision tree can be learned directly from categorical data with no data transform required.
# Many machine learning algorithms cannot operate on label data directly. 
# They require all input variables and output variables to be numeric.
# In general, this is mostly a constraint of the efficient implementation of machine learning algorithms
# rather than hard limitations on the algorithms themselves.
# This means that categorical data must be converted to a numerical form.
To Convert Categorical Data to Numerical Data, there are two steps:
1. Integer Encoding : 
    As a first step, each unique category value is assigned an integer value.
    For example, "Audi" is 1, "BMW” is 2, and "Ford” is 3.
    The integer values have a natural ordered relationship between each other and machine learning algorithms 
    may be able to understand and harness this relationship.
2. One-Hot Encoding: 
    For categorical variables where no such ordinal relationship exists, the integer encoding is not enough.
    In fact, using this encoding and allowing the model to assume a natural ordering between categories may result in poor performance 
    or unexpected results (predictions halfway between categories).
    In this case, a one-hot encoding can be applied to the integer representation. 
    This is where the integer encoded variable is removed and a new binary variable is added for each unique integer value.
    In the "Model" variable example, lets there are 3 categories and therefore 3 binary variables are needed. 
    A “1” value is placed in the binary variable for the model and “0” values for the other model.
    For example:
    Audi, BMW, Ford
    1,    0,    0
    0,    1,    0
    0,    0,    1
    The binary variables are often called “dummy variables” in other fields, such as statistics.
'''

ohe = OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])
# column_trans = ohe.fit_transform(x[['name','company','fuel_type']])

OneHotEncoder()

In [116]:
'''
Column Transformer is a sciket-learn class used to create and apply separate transformers for numerical and categorical data. 
To create transformers we need to specify the transformer object and pass the list of transformations inside a tuple 
along with the column on which you want to apply the transformation. 
'''

column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),
                                      remainder='passthrough')

In [117]:
''' 
Linear regression analysis is used to predict the value of a variable based on the value of another variable.
The variable you want to predict is called the dependent variable. 
The variable you are using to predict the other variable's value is called the independent variable.

This form of analysis estimates the coefficients of the linear equation, 
involving one or more independent variables that best predict the value of the dependent variable. 
Linear regression fits a straight line or surface that minimizes the discrepancies between predicted and actual output values.
There are simple linear regression calculators that use a “least squares” method to discover
the best-fit line for a set of paired data. 
You then estimate the value of X (dependent variable) from Y (independent variable).
'''
lr = LinearRegression()

In [118]:
'''
In ML pipeline means of automating the machine learning workflow by enabling data to be transformed 
and correlated into a model that can then be analyzed to achieve outputs.  
'''
pipe = make_pipeline(column_trans, lr)

In [119]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=42)
pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [120]:
y_pred = pipe.predict(x_test)

In [121]:
r2_score(y_test, y_pred)

0.573149277112929

### Finding the model with a random state of TrainTestSplit where the model was found to give almost 0.88 

In [122]:
'''  
train_test_split splits arrays or matrices into random train and test subsets. 
That means that everytime you run it without specifying random_state, 
you will get a different result, this is not an expected behavior.

With random_state=None , we get different train and test sets across different executions 
and the shuffling process is out of control.

With random_state=0 , we get the same train and test sets across different executions.
With random_state=42, we get the same train and test sets across different executions, 
but in this time, the train and test sets are different from the previous case with random_state=0 .

Many students and practitioners use this number(42) as random state is because it is used by
many instructors in online courses. They often set the random state or numpy seed to number 42
and learners follow the same practice without giving it much thought.

The train and test sets directly affect the model’s performance score. 
Because we get different train and test sets with different integer values for random_state 
in the train_test_split() function, the value of the random state hyperparameter indirectly 
affects the model’s performance score.
'''
scores=[]
for i in range(1000):
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans, lr)
    pipe.fit(x_train, y_train)
    y_pred=pipe.predict(x_test)
#     print(r2_score(y_test, y_pred), i)
    scores.append(r2_score(y_test, y_pred))

In [123]:
np.argmax(scores)

433

In [124]:
scores[np.argmax(scores)]

0.8456515104452564

In [125]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=np.argmax(scores))
# For Linear Regression
lr=LinearRegression()
pipe=make_pipeline(column_trans, lr)
pipe.fit(x_train, y_train)
y_pred=pipe.predict(x_test)
print("For Linear Regression r2_score is : ", r2_score(y_test, y_pred))
'''
r2_score ->  (total variance explained by model) / (total variance).  
variance is a measure of how far observed values differ from the average of predicted values,
'''


For Linear Regression r2_score is :  0.8456515104452564


In [126]:
import pickle

In [127]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [128]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti', 2019,100,'Petrol']], columns=['name','company','year','kms_driven','fuel_type']))

array([459113.49353657])

In [129]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti', 2017,100,'Petrol']], columns=['name','company','year','kms_driven','fuel_type']))

array([394402.96645257])

In [130]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti', 2015,1000,'Petrol']], columns=['name','company','year','kms_driven','fuel_type']))

array([329297.28320999])

In [131]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti', 2015,500000,'Petrol']], columns=['name','company','year','kms_driven','fuel_type']))

array([110205.14639507])